## import and stuff

In [1]:
from torrent_file_parser import TorrentFile
from bencoding import Encoder, Decoder
from hashlib import sha1
import socket
from random import randrange
import struct
from urllib.parse import urlsplit
from threading import Thread
from queue import Queue

torrent_file = 'files/file2.torrent'

In [2]:
def run_async(f, daemon=False):
    def wrapped_f(q, *args, **kwargs):
        ret = f(*args, **kwargs)
        q.put(ret)
    
    def wrap(*args, **kwargs):
        q = Queue()
        t = Thread(target=wrapped_f, args=(q, ) + args, kwargs=kwargs)
        t.daemon = daemon
        t.start()
        t.result_queue = q
        return t
    
    return wrap

# read meta_data from the file

In [3]:
with open(torrent_file, 'rb') as f:
    meta_data = Decoder(f.read()).decode()
    

info =  Encoder(meta_data[b'info']).encode()
info_hash = sha1(info).digest()

torrent_object = TorrentFile(torrent_file)
trackers = torrent_object.announce_list

In [4]:
def create_req_struct(protocol_id=0X41727101980, transcation_id=randrange(0, 65535), action=0):
    #protocol_id is const, transcation id  is random num , action 0 = connect
    try:
        req_struct = struct.pack(">QLL", protocol_id, action, transcation_id)
    except Excpetion:
        raise Excpetion("the req struct creation failed")
    return req_struct


In [5]:
def parse_udp_url(url):
    try:
        scheme, netlok, params, path, _  = urlsplit(url)
    except Exception:
        raise Exception(f"{url} parse url failes")
    if scheme != 'udp':
        raise Excpetion("the url not relevant for udp tracker request")
    url, port = netlok.split(':')
    try:
        port = int(port)
    except Excpetion:
        raise Excpetion("the port number have problem")
    return url, port

In [6]:
def parse_res(packet):
    try:
        req_response = struct.unpack(">LLQ", packet)
    except Excpetion:
        raise Excption("the unpack connect response failed")
    return req_response

In [7]:
#create connect req
@run_async
def tracker_connect_req(sock, url):
    #the funcion ask for socket connection on udp from udp tracket url 
    req_struct = create_req_struct()
    address = parse_udp_url(url)
    sock.sendto(req_struct, address)
    res = sock.recv(16)
    return parse_res(res)
    


In [16]:
trackers[0]

'http://inferno.demonoid.me:3414/announce'

In [8]:
@run_async
def udp_tracker_req(torrent_file):
    pass

In [9]:
@run_async
def http_tracker_connection(url, info_hash):
    

(0, 46280, 2786357492193302238)

NoneType